In [1]:
import cv2
import numpy as np
cap = cv2.VideoCapture("rtsp://192.168.137.159:554/mjpeg/1")
# 设置摄像头参数
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
# cap.set(cv2.CAP_PROP_FPS, 10)


In [2]:
def perspective_transform(image, src_points):
    # 定义目标图像大小
    target_width = 480
    target_height = 640

    # 定义目标四个角点
    dst_points = np.array([[0, 0], [target_width - 1, 0], [target_width - 1, target_height - 1], [0, target_height - 1]], dtype=np.float32)

    # 计算透视变换矩阵
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # 应用透视变换
    warped_image = cv2.warpPerspective(image, matrix, (target_width, target_height))

    return warped_image

In [3]:
def sort_points(contourroad):
    list = [[0,0],[0,0],[0,0],[0,0]]
    # 计算每个子数组中两个数的和
    sums = np.sum(contourroad, axis=2)
    # 根据和的大小进行排序
    minpointidx = np.argmin(sums[:, 0])
    list[0] = contourroad[minpointidx]
    contourroad = np.delete(contourroad, minpointidx, axis=0)
    sums = np.sum(contourroad, axis=2)
    maxpointidx = np.argmax(sums[:, 0])
    list[2] = contourroad[maxpointidx]
    contourroad = np.delete(contourroad, maxpointidx, axis=0)
    if contourroad[0][0][0]-contourroad[0][0][1] >= 0:
        list[1] = contourroad[0]
        list[3] = contourroad[1]
    else:
        list[3] = contourroad[0]
        list[1] = contourroad[1]
    return np.array(list).astype(np.float32)

In [4]:
def Splitarray(array):
    # 初始化数组列表来存储分割后的数组
    split_arrays = []
    current_subarray = []

    # 遍历数组并根据间隔大于100的条件进行分割
    for num in array:
        if current_subarray and num - current_subarray[-1] > 100:
            split_arrays.append(np.array(current_subarray))
            current_subarray = [num]
        else:
            current_subarray.append(num)

    # 添加最后一个子数组
    if current_subarray:
        split_arrays.append(np.array(current_subarray))

    return split_arrays

In [5]:
import math
class FIFO:
  """
  FIFO 类，用于保存坐标信息
  """

  def __init__(self, max_size):
    """
    初始化 FIFO 类

    Args:
      max_size: FIFO 队列的最大长度
    """

    self.max_size = max_size
    self.queue = []

  def add(self, point):
    """
    将坐标添加到 FIFO 队列中

    Args:
      x: X 坐标
      y: Y 坐标
    """

    # 如果队列已满，则弹出第一个元素
    if len(self.queue) == self.max_size:
      self.queue.pop(0)

    # 将新元素添加到队列末尾
    self.queue.append(point)

  def get(self):
    """
    从 FIFO 队列中获取坐标

    Returns:
      一个包含 X 坐标和 Y 坐标的元组
    """

    if len(self.queue) == 0:
      return None

    # 返回第一个元素
    return self.queue[0]

  def is_full(self):
    """
    判断 FIFO 队列是否已满

    Returns:
      True 如果队列已满，False 如果队列未满
    """

    return len(self.queue) == self.max_size

  def is_empty(self):
    """
    判断 FIFO 队列是否为空

    Returns:
      True 如果队列为空，False 如果队列非空
    """

    return len(self.queue) == 0

In [6]:

def findcar(image):
    # 将图像转换为 HSV 颜色空间
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 定义蓝色颜色的范围
    lower_blue = np.array([100, 100, 100])
    upper_blue = np.array([120, 255, 255])

    # 创建一个掩膜来提取蓝色区域
    mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

    # 寻找蓝色区域的轮廓
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 遍历轮廓并找到最大的轮廓
    max_area = 0
    max_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_contour = contour

    # 计算蓝色区域的外接框
    x, y, w, h = cv2.boundingRect(max_contour)

    return [[x,y],[x+w,y+h]]

def findcar_2(image,template):
    # 使用 TM_CCOEFF_NORMED 方法进行模板匹配
    res = cv2.matchTemplate(image, template, cv2.TM_SQDIFF_NORMED)

    # 找到最匹配的位置
    min_val, max_val, max_loc, min_loc = cv2.minMaxLoc(res)

    # 获取最佳匹配位置坐标
    car_top_left = max_loc
    h, w = template.shape[:2]
    car_bottom_right = (car_top_left[0] + w, car_top_left[1] + h)
    return [car_top_left,car_bottom_right]

In [7]:
contourroad = np.array([[[203.,  38.]],

                        [[435.,  45.]],

                        [[569., 418.]],

                        [[ 87., 427.]]], dtype=np.float32)

template_car = cv2.imread('../trafficidf/car.png')

isfindcar = False

speed_fifo = FIFO(5)

while True:
    
    ret, frame = cap.read()
    
    # 转换为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 高斯模糊
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny边缘检测
    edges = cv2.Canny(blurred, 50, 150)

    # 寻找轮廓
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 遍历轮廓并找到白色大块方形
    # 遍历轮廓并找到试卷

    for contour in contours:
        # 计算轮廓的面积
        area = cv2.contourArea(contour)

        # 过滤掉面积小的轮廓
        if area < 10000:
            continue

        # 拟合四边形
        approx = cv2.approxPolyDP(contour, cv2.arcLength(contour, True) * 0.02, True)
            # 对轮廓进行多边形逼近，拟合成四边形
        # 判断是否是四边形
        if len(approx) == 4:
        # 返回试卷的四个角的位置
            # 绘制拟合后的四边形
            # cv2.drawContours(frame, [approx], -1, (0, 255, 0),3)
            contourroad = approx
    
    sortpoints = sort_points(contourroad)
    roadimage = perspective_transform(frame,sortpoints)

    
    gray_roadimage = cv2.cvtColor(roadimage, cv2.COLOR_BGR2GRAY)

    car_st,car_end = findcar(roadimage)
    car_center = np.add(car_st,car_end)/2
    speed_fifo.add(car_center)
    
    if speed_fifo.is_full:
        car_move = car_center - speed_fifo.get()
        speed_car = math.sqrt(car_move[0]**2 +car_move[1]**2)

        cv2.putText(roadimage,f'speed:{speed_car}',(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    # 在原始图像上绘制矩形标记匹配位置
    cv2.rectangle(roadimage, car_st, car_end, (0, 255, 0), 2)

    vertical_sum = np.sum(gray_roadimage, axis=0)
    # 计算竖向求和结果的中位数
    median_value = np.median(vertical_sum)-40000

    # 找到数值大于等于中位数的列的索引
    valid_indices = np.where(vertical_sum <= median_value)[0]
    if valid_indices is not None:
        split_array = Splitarray(valid_indices)
        if len(split_array) == 3:
            roadline0 = [split_array[0][0],split_array[0][-1]]
            roadline1 = [split_array[1][0],split_array[1][-1]]
            roadline2 = [split_array[2][0],split_array[2][-1]]

            for line in [roadline0,roadline1,roadline2]:
                cv2.line(roadimage, (line[0]+5, 0), (line[0], roadimage.shape[0]), (0,0,255), thickness=2)
                cv2.line(roadimage, (line[1]-5, 0), (line[1], roadimage.shape[0]), (0,0,255), thickness=2)

    horizon_sum = np.sum(gray_roadimage, axis=1)
    # 计算竖向求和结果的中位数
    median_value_h = np.median(horizon_sum)-35000

    valid_indices_h = np.where(horizon_sum <= median_value_h)[0]
    if valid_indices_h is not None:
        roadline4 = [valid_indices_h[0],valid_indices_h[-1]]

        cv2.line(roadimage, (0, roadline4[0]), (roadimage.shape[1],roadline4[0]), (0,0,255), thickness=2)
        cv2.line(roadimage, (0, roadline4[1]), (roadimage.shape[1],roadline4[1]), (0,0,255), thickness=2)
    
    



    cv2.imshow('result',frame)
    # 按下esc键退出
    if cv2.waitKey(10) & 0xFF == 27:
        break
cv2.destroyAllWindows()

In [66]:
np.add(car_st,car_end)/2

array([352.5, 426.5])

In [106]:
valid_indices_h

array([573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598,
       599, 600, 601, 602, 603, 604, 605, 606, 607, 608], dtype=int64)

In [8]:
cap.release()
